In [1]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd
import os.path
from os import path

In [2]:
def combine_team_games(df, keep_method='home'):
    '''Combine a TEAM_ID-GAME_ID unique table into rows by game. Slow.

        Parameters
        ----------
        df : Input DataFrame.
        keep_method : {'home', 'away', 'winner', 'loser', ``None``}, default 'home'
            - 'home' : Keep rows where TEAM_A is the home team.
            - 'away' : Keep rows where TEAM_A is the away team.
            - 'winner' : Keep rows where TEAM_A is the losing team.
            - 'loser' : Keep rows where TEAM_A is the winning team.
            - ``None`` : Keep all rows. Will result in an output DataFrame the same
                length as the input DataFrame.
                
        Returns
        -------
        result : DataFrame
    '''
    # Join every row to all others with the same game ID.
    joined = pd.merge(df, df, suffixes=['_A', '_B'],
                      on=['SEASON_ID', 'GAME_ID', 'GAME_DATE'])
    # Filter out any row that is joined to itself.
    result = joined[joined.TEAM_ID_A != joined.TEAM_ID_B]
    # Take action based on the keep_method flag.
    if keep_method is None:
        # Return all the rows.
        pass
    elif keep_method.lower() == 'home':
        # Keep rows where TEAM_A is the home team.
        result = result[result.MATCHUP_A.str.contains(' vs. ')]
    elif keep_method.lower() == 'away':
        # Keep rows where TEAM_A is the away team.
        result = result[result.MATCHUP_A.str.contains(' @ ')]
    elif keep_method.lower() == 'winner':
        result = result[result.WL_A == 'W']
    elif keep_method.lower() == 'loser':
        result = result[result.WL_A == 'L']
    else:
        raise ValueError(f'Invalid keep_method: {keep_method}')
    return result
    

In [9]:
attempt = pd.read_csv("ten_day-19-20.csv",index_col=[0])
# attempt.set_index('SEASON_ID')
# attempt = pd.read_csv("correct_dates.csv",index_col=[0])
attempt = attempt.drop(['PTS','FGM','FGA','FG_PCT','FG3M','FG3A','FG3_PCT','FTM','FTA','FT_PCT','OREB','DREB','REB','AST','STL','BLK','TOV','PF', 'PLUS_MINUS'],axis=1)
# attempt


In [4]:
# first = attempt.loc[attempt['GAME_ID'] == 21800020]
first = attempt.loc[attempt['GAME_ID'] == 21700009]
first

,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,AV_PTS,AV_FGM,...,AV_FT_PCT,AV_OREB,AV_DREB,AV_REB,AV_AST,AV_STL,AV_BLK,AV_TOV,AV_PF,AV_PLUS_MINUS
SEASON_ID,,,,,,,,,,,,,,,,,,,,,
22017,1610612737,ATL,Atlanta Hawks,21700009,2017-10-18,ATL @ DAL,W,241,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22017,1610612742,DAL,Dallas Mavericks,21700009,2017-10-18,DAL vs. ATL,L,240,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# uses combine function
count = 0
for row in attempt.iterrows():
    if (count == 0):
        catch = attempt.loc[attempt['GAME_ID'] == row[1]['GAME_ID']]
        catch = pd.DataFrame(catch)
        combine = combine_team_games(catch)
        combine.to_csv('combined_19-20.csv', index=False)
        count = count + 1
    else: 
        old_df = pd.read_csv('combined_19-20.csv')
        catch = attempt.loc[attempt['GAME_ID'] == row[1]['GAME_ID']]
        catch = pd.DataFrame(catch)
        combine = combine_team_games(catch)
        new_df = old_df.append(combine)
        new_df.to_csv('combined_19-20.csv', index=False)
#         count = count + 1
#     if (count == 5):
#         break

In [11]:
clean = pd.read_csv('combined_19-20.csv')

In [12]:
# drops duplicates, sort by game date, and replace W with 1 and L with 0
cleaned = clean.drop_duplicates(subset='GAME_ID')
cleaned = cleaned.sort_values('GAME_DATE')
cleaned['WL_A'] = cleaned['WL_A'].replace(['W','L'],[1,0])
cleaned['WL_B'] = cleaned['WL_B'].replace(['W','L'],[1,0])

In [13]:
cleaned.to_csv('combined_CLEANED_2019-2020.csv', index=False)

In [ ]:
#hi